**About** : This notebook is used to infer models.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
cd ../src/

/home/tviel/work/kaggle_birdclef_2024/src


/home/tviel/.local/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


### Imports

In [3]:
import os
import gc
import sys
import cv2
import glob
import json
import torch
import shutil
import librosa
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import torch.nn.functional as F
import matplotlib.pyplot as plt

from tqdm import tqdm
from joblib import Parallel, delayed

In [4]:
from util.logger import Config
from util.metrics import macro_auc
from util.torch import load_model_weights

from data.dataset import WaveInfDataset
from data.preparation import prepare_data, prepare_folds
from data.processing import create_target_path, ProgressParallel, get_load_librosa_save_h5py

from model_zoo.models import define_model
from inference.predict import predict

from params import CLASSES

/home/tviel/.local/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


### Params

In [10]:
EVAL = True

In [11]:
if EVAL:
    DATA_PATH = "../input/train_audio/"
else:
    DATA_PATH = "../input/test_soundscapes/"

SAVE_FOLDER = "../output/tmp/"

if os.path.exists(SAVE_FOLDER):
    shutil.rmtree(SAVE_FOLDER)
os.makedirs(SAVE_FOLDER)

In [12]:
BATCH_SIZE = 32
USE_FP16 = True
NUM_WORKERS = 2

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

DURATION = 5
SR = 32000

In [83]:
FOLD = 0 if EVAL else "fullfit_0"

EXP_FOLDERS = [
    # ("../logs/2024-04-12/8/", [FOLD]),   # 0.978 / 0.972
    # ("../logs/2024-04-15/0/", [FOLD]),
    # ("../logs/2024-04-16/15/", [FOLD]),  # 0.983 / 0.977
    # ('../logs/2024-04-16/17/', [FOLD]),  # 0.987 / 0.982
]

In [84]:
# from util.logger import upload_to_kaggle

# upload_to_kaggle(
#     [f for f, _ in EXP_FOLDERS],
#     directory="../output/dataset_1/",
#     dataset_name="BirdCLEF 2024 Weights 1",
#     update_folders=True
# )

### Preprocessing

In [85]:
if EVAL:
    df = pd.DataFrame({"path": glob.glob(DATA_PATH + "*/*")})
    df["id"] = df["path"].apply(lambda x: x.split("/")[-1][:-4])

    folds = pd.read_csv('../input/folds_4.csv')
    folds['id'] = folds['filename'].apply(lambda x: x.split('/')[-1][:-4])
    df = df.merge(folds)
    df = df[df['fold'] == 0].reset_index(drop=True)

    df["primary_label"] = df["path"].apply(lambda x:  x.split('/')[-2])
else:
    df = pd.DataFrame({"path": glob.glob(DATA_PATH + "*")})
    df["id"] = df["path"].apply(lambda x: x.split("/")[-1][:-4])

df["duration"] = df["path"].apply(lambda x: librosa.get_duration(path=x))

df['slice'] = df['duration'].apply(lambda x: [(DURATION * i * SR, DURATION * (i + 1) * SR) for i in range(int(np.ceil(x / DURATION)))])
df = df.explode("slice")

df["t_end"] = DURATION
df['t_end'] = df[["id", "t_end"]].groupby("id").cumsum()

if EVAL:
    df = df[df['t_end'] == 5]

### Models

In [86]:
models = []

for exp_folder, folds in EXP_FOLDERS:
    models_ = []
    config = Config(json.load(open(exp_folder + "config.json", "r")))

    model = define_model(
        config.name,
        config.melspec_config,
        head=config.head,
        aug_config=config.aug_config,
        num_classes=config.num_classes,
        n_channels=config.n_channels,
        drop_rate=config.drop_rate,
        drop_path_rate=config.drop_path_rate,
        increase_stride=config.increase_stride,
        verbose=True,
        pretrained=False
    )
    model = model.to(DEVICE).eval()
    
    for fold in folds:
        weights = exp_folder + f"{config.name}_{fold}.pt"
        model = load_model_weights(model, weights, verbose=config.local_rank == 0)
        models_.append(model)
        
    models.append(models_)


 -> Loading encoder weights from ../logs/2024-04-12/8/tf_efficientnetv2_s_0.pt



In [87]:
dataset = WaveInfDataset(
    df,
    normalize=config.normalize,
    max_len=config.melspec_config["sample_rate"] * config.duration,
)

In [88]:
preds = predict(
    model,
    dataset,
    config.loss_config,
    batch_size=BATCH_SIZE,
    device=DEVICE,
    use_fp16=USE_FP16,
    num_workers=NUM_WORKERS,
)

  0%|          | 0/191 [00:00<?, ?it/s]

In [89]:
if EVAL:
    auc = macro_auc(df["primary_label"].values.tolist(), preds)
    print(f'Fold 0 AUC: {auc:.3f}')

Fold 0 AUC: 0.978


In [90]:
if EVAL:
    from data.preparation import prepare_data
    from data.dataset import WaveDataset

    df_ = prepare_data()
    df_ = df_[df_['fold'] == 0].reset_index(drop=True)
    df = df.merge(df_[["filename", "secondary_labels"]], how="left")
    df['path_ft'] = df['path']
    dataset = WaveDataset(df)

    y, y_s = dataset.get_targets()

    auc = macro_auc(y, preds)
    print(f'Fold 0 AUC:\t {auc:.3f}')
    
    auc = macro_auc(y_s, preds)
    print(f'Fold 0 AUC_s:\t {auc:.3f}')

Fold 0 AUC:	 0.978
Fold 0 AUC_s:	 0.972


In [19]:
sub = pd.DataFrame((df["id"] + "_" + df["t_end"].astype(str)).values, columns=['row_id'])
sub.loc[:, CLASSES] = preds
# sub.to_csv('submission.csv', index=False)
sub.head()

,row_id,asbfly,ashdro1,ashpri1,ashwoo2,asikoe2,asiope1,aspfly1,aspswi1,barfly1,...,whbwoo2,whcbar1,whiter2,whrmun,whtkin2,woosan,wynlau1,yebbab1,yebbul3,zitcis1
0,XC199339_5,0.016983,0.001289,0.002068,0.000238,0.002090,0.000421,0.001053,0.000077,0.000750,...,0.000419,0.000175,0.002165,0.000602,0.003351,0.035828,0.000145,0.000296,0.000225,0.052917
1,XC199339_10,0.074524,0.000785,0.000497,0.000028,0.001554,0.000265,0.000274,0.000021,0.000789,...,0.000542,0.000171,0.000573,0.000137,0.002209,0.006798,0.000089,0.000181,0.000278,0.001629
2,XC199339_15,0.000373,0.000134,0.000161,0.000041,0.000330,0.000016,0.000050,0.000003,0.000031,...,0.000162,0.000004,0.000202,0.000056,0.000391,0.004101,0.000012,0.000020,0.000014,0.009857
3,XC757943_5,0.000283,0.000227,0.000900,0.000009,0.000859,0.000158,0.000081,0.000014,0.000188,...,0.000199,0.000074,0.000873,0.000085,0.001095,0.009933,0.000012,0.000081,0.000073,0.004280
4,XC246306_5,0.003351,0.001616,0.014229,0.000251,0.022415,0.000811,0.001661,0.000735,0.003651,...,0.006145,0.004906,0.002150,0.001554,0.006771,0.007374,0.001184,0.001525,0.001897,0.006931


Done ! 